In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.insert(0, '../')

from llmagent.privatestore import WebStore, WebConf
from llmagent.llmapi import QianfanEmbedding
from llmagent.secret import AK_SK
keys = AK_SK('../llmagent/secret/keystore/qianfan.keys')
os.environ["QIANFAN_ACCESS_KEY"] = keys.get_ak()
os.environ["QIANFAN_SECRET_KEY"] = keys.get_sk()

USER_AGENT environment variable not set, consider setting it to identify your requests.
[WARNING][2024-10-22 22:03:11.584] redis_rate_limiter.py:21 [t:8703163072]: no redis installed, RedisRateLimiter unavailable


In [3]:
# from llmagent.llmapi import QianfanLLM, QianfanEmbedding, model_spec

# llm = QianfanLLM(model_spec=model_spec.Speed128K, temperature=0.5)
embedding = QianfanEmbedding()

In [21]:
webstore = WebStore("llmtutorial", "../vectorstore/llmtutorial", emb_func=embedding, extra_config=WebConf(
    root_url="https://aitutor.liduos.com/",
    # exclude_tags=['li'],
    limit=10,
))

In [25]:
webstore._extra_config

WebConf(batch_size=16, chunk_size=300, chunk_overlap=100, root_url='https://aitutor.liduos.com/', exclude_tags=[], limit=10, header_template={}, verify_ssl=False, proxies=None, continue_on_failure=True, autoset_encoding=True, encoding=None, default_parser='html.parser', bs_kwargs=None)

In [23]:
webstore.del_docment("https://aitutor.liduos.com/01-llm/01-1.html")

In [26]:
webstore.reload()

/Users/flybywindwen/miniconda3/envs/llm-wechat/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/flybywindwen/miniconda3/envs/llm-wechat/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
2024-10-22 22:18:09.547 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 0 to 16
2024-10-22 22:18:09.918 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 16 to 32
2024-10-22 22:18:10.132 | INFO     | llmagent.privatestore.base:__load__:51 - Adding document

In [27]:
webstore.collection_count()

1483

In [29]:
webstore._documents['https://aitutor.liduos.com/01-llm/01-2.html'].page_content

'Title    你好, ChatGPT (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已经开售！推荐正在学习AI应用开发的朋友购买阅读！   你好, ChatGPT  ChatGPT 是OpenAI开发的人工智能聊天机器人程序，于2022年11月推出。该程序使用基于 GPT-3.5、GPT-4 架构的大语言模型并以强化学习训练。ChatGPT目前仍以文字方式交互，而除了可以用人类自然对话方式来交互，还可以用于甚为复杂的语言工作，包括自动生成文本、自动问答、自动摘要等多种任务。  ChatGPT的诞生  演进过程     总结    训练  训练有四个主要阶段：预训练、有监督微调、奖励建模、强化学习  Pretraining 预训练    Supervised Finetuning 监督微调    Reward Modeling 奖励建模    Reinforcement Learning 强化学习    特点    局限    参考链接        results matching " "     No results matching " "         (01-1.html)  (01-1.html)   (01-3.html)  (01-3.html)'

## 构建Prompt

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
# 感觉需要精简一下问题模版。现在llm中会自动记录之前的问答历史，但是如果增加了context，那么每次都需要api处理多个相同的context，感觉非常浪费
prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "你是一个解答用户问题的assistant，可以根据从语料库中召回的context文本回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留 source 后面的url链接。\n以下是context文本：{context}，\n问题是：{question}"),
            ("assistant", "根据context文本，我认为答案是："),
        ]
    )
# 创建 ChatPromptTemplate，直接在模板中包含历史记录变量
template = ChatPromptTemplate.from_messages(
    [("system", "This is the conversation so far:\n{history}"), ("human", "{input}")]
)

# 假设有一些聊天历史记录
chat_history = [
    HumanMessage(content="Hi, how are you?"),
    AIMessage(content="I'm good, thanks! How can I help you today?"),
    HumanMessage(content="Tell me a joke."),
    AIMessage(content="Why don't scientists trust atoms? Because they make up everything!")
]

# 将历史记录转换为字符串，控制需要传递的历史消息长度
recent_history = "\n".join([f"{message.type}: {message.content}" for message in chat_history])

# 将裁剪后的历史和用户输入传递给模板
prompt = template.format_messages(
    history=recent_history,
    input="Can you explain that joke?"
)

for message in prompt:
    print(f"{message.type}: {message.content}")



system: This is the conversation so far:
human: Hi, how are you?
ai: I'm good, thanks! How can I help you today?
human: Tell me a joke.
ai: Why don't scientists trust atoms? Because they make up everything!
human: Can you explain that joke?


## test before

In [26]:
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader
from urllib.parse import urljoin, urlparse


In [25]:
url = "https://aitutor.liduos.com/"

# html_loader = AsyncHtmlLoader(web_path=url)
# document = html_loader.load()[0]

In [20]:
# from langchain.document_loaders import WebBaseLoader

# # def get_same_domain_links(url):
#     # Load the webpage
# loader = WebBaseLoader(url)
# document = loader.load()[0] # 返回的是markdown格式的文档
# url = "https://aitutor.liduos.com/"


In [77]:
from typing import Iterator
from bs4 import BeautifulSoup
from loguru import logger
from langchain_community.document_transformers import BeautifulSoupTransformer

def _build_metadata(soup, url: str) -> dict:
    """Build metadata from BeautifulSoup output."""
    metadata = {"source": url}
    if title := soup.find("title"):
        metadata["title"] = title.get_text()
    if description := soup.find("meta", attrs={"name": "description"}):
        metadata["description"] = description.get("content", "No description found.")
    if html := soup.find("html"):
        metadata["language"] = html.get("lang", "No language found.")
    return metadata

class WebRecursiveLoader(WebBaseLoader):
    def __init__(self, root_url:str, depth: int = 1, unwanted_tags:List[str] = [], **kwargs):
        super().__init__(web_path=root_url, **kwargs)
        self.domain = urlparse(root_url).netloc
        self.depth = depth
        self.bs_transformer = BeautifulSoupTransformer()
        self.unwanted_tags = list(set(unwanted_tags).union(set(["script", "style"])))

    def lazy_load(self) -> Iterator[Document]:
        loaded_set = set()
        def __inner_load__(url: str, depth: int):
            if depth < 0:
                return
            soup:BeautifulSoup = self._scrape(url, bs_kwargs=self.bs_kwargs)
            # text = soup.get_text(**self.bs_get_text_kwargs)
            text = str(soup)
            metadata = _build_metadata(soup, url)
            yield Document(page_content=text, metadata=metadata)
            loaded_set.add(url)
            links = soup.find_all("a", href=True)
            for link in links:
                full_url = urljoin(url, link["href"])
                parsed_url = urlparse(full_url)
                if parsed_url.netloc == self.domain and full_url not in loaded_set:
                    yield from __inner_load__(full_url, depth - 1)
        yield from __inner_load__(self.web_path, self.depth)
    
    def load(self) -> List[Document]:
        return self.bs_transformer.transform_documents(list(self.lazy_load()), unwanted_tags=self.unwanted_tags)


In [78]:
loader = WebRecursiveLoader(root_url=url, depth=2, unwanted_tags=["li"], requests_per_second=10)
documents = loader.load()

In [79]:
len(documents)

42

In [82]:
documents[14].page_content

'Title    LlamaIndex介绍 (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已经开售！推荐正在学习AI应用开发的朋友购买阅读！   LlamaIndex介绍  LlamaIndex(也称为GPT Index)是一个用户友好的接口，它将外部数据连接到大型语言模型(Large Language Models, llm)。它提供了一系列工具来简化流程，包括可以与各种现有数据源和格式(如api、pdf、文档和SQL)集成的数据连接器。此外，LlamaIndex为结构化和非结构化数据提供索引，可以毫不费力地与大语言模型一起使用。  包括列表索引、矢量存储索引、树索引和关键字表索引的分解，以及图索引、Pandas索引、SQL索引和文档摘要索引。  如果可用的tokens不多，则无法在prompt中输入更大的数据集，这可能会限制对模型的操作。使用LlamaIndex，可以为各种数据集(如文档、pdf和数据库)建立索引，然后轻松地查询它们以查找所需的信息。可以直接向知识库、Slack和其他通信工具以及数据库和几乎所有SaaS内容提出复杂的问题，而无需以任何特殊方式准备数据。       results matching " "     No results matching " "         (03-2.html)  (03-2.html)'

In [83]:
documents[14].metadata

{'source': 'https://aitutor.liduos.com/03-llamaIndex/03-1.html',
 'title': 'LlamaIndex介绍 · LLM 应用开发实践笔记',
 'description': '',
 'language': 'zh-hans'}